# Interactive Visualization of Daytimes and Workingtimes in Europe

## Data

### EU countries to capitals and GeoPandas data

In [1]:
import json

from src.geo_utils import load_eu_countries_as_geopandas, get_eu_capitals

eu_capitals = get_eu_capitals()
eu_gpd = load_eu_countries_as_geopandas()
eu_iso_to_geo = eu_gpd.loc[:, ['iso_a3', 'geometry']]

### Sunrise/Sunset (UTC) for the year 2022

In [2]:
from src.sun_data import get_sunset_sunrise_data
sun_data_df = get_sunset_sunrise_data(year=2022, country_capital=eu_capitals)

Could not process day 2022-06-16 00:00:00


In [3]:
import geopandas as gpd
sun_data_gpd = gpd.GeoDataFrame(sun_data_df.merge(eu_iso_to_geo, on='iso_a3', how='left').set_index('iso_a3'))

## Interactive visualization with Panel/Bokeh

In [46]:
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
import json
import panel as pn
from datetime import datetime

data_field = 'sunrise_UTC'
bokeh_tools = 'wheel_zoom,pan,reset'

def get_bokeh_geodata_source(gpd_df):
    json_data = json.dumps(json.loads(gpd_df.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(data):
    geo_data_source = get_bokeh_geodata_source(data)

    values = data[data_field]
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = values.min(), high = values.max())
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                         location=(0,0), orientation='horizontal')

    p = figure(plot_height = 400, plot_width=850, toolbar_location='right', tools=bokeh_tools)
    p.patches('xs','ys', source=geo_data_source,
              fill_color={'field': data_field, 'transform': color_mapper},
              line_color='blue',
              line_width=0.5,
              fill_alpha=0.8)
    p.add_layout(color_bar, 'below')
    return p

In [47]:
def map_visualization():
    map_pane = pn.pane.Bokeh(width=400)
    default_date = datetime.strptime('2022-01-01', '%Y-%m-%d').date()
    selected_date = pn.widgets.DatePicker(name='Date Picker', value=default_date)
    def update_map(event):
        d = selected_date.value
        selected_sundata = sun_data_gpd.query(f'day == {d.day} & month == {d.month} & year == {d.year}')
        map_pane.object = bokeh_plot_map(selected_sundata)
    selected_date.param.watch(update_map, 'value')
    selected_date.param.trigger('value')
    return pn.Column(selected_date, map_pane)

app = map_visualization()
app.show()

Launching server at http://localhost:1477
